<a href="https://colab.research.google.com/github/wamaw123/Biomedical-Data-Analytics-with-Python/blob/main/Foundations%20of%20Data%20Analytics/Disease_Prediction_and_Prevention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Analytics with Python
By : [Abderrahim Benmoussa, Ph.D. ](https://https://github.com/wamaw123)

Project's on Github : https://github.com/wamaw123/Biomedical_Data_analysis

# Disease Prediction and Prevention

In this notebook, we aim to predict the 10-year risk of future coronary heart disease (CHD) using the Framingham Heart Study dataset. This will be a binary classification task, where `1` indicates the risk of CHD, and `0` indicates no risk. To do so we will explore the dataset and go on to test a simple logistic regression model first. We will then compare different models and finally optimize the model to get the best predictive accuracy.

## Step 1: Install and Import Libraries

In this step, we will install and import necessary libraries for our analysis.
- `pandas` and `numpy` for data manipulation
- `seaborn` and `matplotlib` for data visualization
- `scikit-learn` for building and evaluating the machine learning model

In [ ]:
# Install necessary packages
!pip install pandas numpy scipy statsmodels patsy dtale scikit-learn pandas_profiling pycaret

# Import necessary libraries

## Data Manipulation
import pandas as pd   # Essential for data manipulation and mathematical operations.
import numpy as np    # Used for array-based operations and mathematical functions.

## Visualization
import matplotlib.pyplot as plt  # Fundamental plotting library.
import seaborn as sns            # Builds on top of matplotlib for more advanced visualizations.

## Statistical Testing, modeling and data preparation
from scipy import stats           # Library for scientific and technical computing.
import statsmodels.api as sm      # Provides classes and functions for the estimation of many different statistical models.
import statsmodels.formula.api as smf  # Formula-based API for the statsmodels library.
from sklearn.model_selection import train_test_split  # Import train_test_split function to split data into training and testing sets.
from sklearn.preprocessing import StandardScaler  # Import StandardScaler to standardize features by removing the mean and scaling to unit variance.
from sklearn.linear_model import LogisticRegression  # Import LogisticRegression to perform logistic regression.
from sklearn.metrics import classification_report, accuracy_score  # Import classification_report to build a text report showing the main classification metrics, and accuracy_score to compute the accuracy of the algorithm.
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV

## Interactive Exploration
from collections import Counter
import pandas_profiling as pp

#Automated Ml
from pycaret.classification import *



## Step 2: Load the Dataset

Next we load the week 2 dataset directly from GitHub and set it into a Pandas dataframe


In [ ]:
# Fetch the dataset from GitHub
url = "https://raw.githubusercontent.com/wamaw123/Biomedical-Data-Analytics-with-Python/afab193c5cb3d6878755c4d12e8baa821a8ab054/Datasets/23/framingham.csv"
df = pd.read_csv(url)
df.head()

#About the dataset

## Source
The dataset is publicly available on the Kaggle website, and it is from an ongoing cardiovascular study on residents of the town of Framingham, Massachusetts. The classification goal is to predict whether the patient has a 10-year risk of future coronary heart disease (CHD). The dataset provides the patients’ information. It includes over 4,000 records and 15 attributes.

## Variables
Each attribute is a potential risk factor. There are both demographic, behavioral, and medical risk factors.

### Demographic:
- **Sex:** male or female (Nominal)
- **Age:** Age of the patient; (Continuous - Although the recorded ages have been truncated to whole numbers, the concept of age is continuous)

### Behavioral:
- **Current Smoker:** whether or not the patient is a current smoker (Nominal)
- **Cigs Per Day:** the number of cigarettes that the person smoked on average in one day. (Can be considered continuous as one can have any number of cigarettes, even half a cigarette.)

### Medical (history):
- **BP Meds:** whether or not the patient was on blood pressure medication (Nominal)
- **Prevalent Stroke:** whether or not the patient had previously had a stroke (Nominal)
- **Prevalent Hyp:** whether or not the patient was hypertensive (Nominal)
- **Diabetes:** whether or not the patient had diabetes (Nominal)

### Medical (current):
- **Tot Chol:** total cholesterol level (Continuous)
- **Sys BP:** systolic blood pressure (Continuous)
- **Dia BP:** diastolic blood pressure (Continuous)
- **BMI:** Body Mass Index (Continuous)
- **Heart Rate:** heart rate (Continuous - In medical research, variables such as heart rate, though in fact discrete, yet are considered continuous because of a large number of possible values.)
- **Glucose:** glucose level (Continuous)

### Predict variable (desired target):
- **10-year risk of coronary heart disease CHD (binary:** "1" means "Yes," "0" means "No")

## Step 3: Data Preprocessing

We will perform initial data preprocessing such as handling missing values. This step is crucial to ensure the quality and reliability of our machine learning model. There are many ways to deal with missing values for instance. Those can be droped or inputed in different ways. Let's first check what missing values we have.

In [ ]:
# Checking for missing values
# isnull() returns a DataFrame where each cell is either True or False depending on that cell's null status.
# sum() will then sum the True values (count of missing values) for each column.
print(df.isnull().sum())

# Calculate the percentage of missing values for each column
missing_percentage = (df.isnull().sum() / len(df)) * 100

# Display the percentage of missing values for each column
print(f"Percentage of missing values for each column:\n{missing_percentage}")

# Visualize the missing values as a heatmap
plt.figure(figsize=(10, 6))  # Set the figure size
sns.heatmap(df.isnull(),     # Provide DataFrame with null-status information
            cbar=False,      # Do not draw a color bar
            cmap='viridis')  # Use the viridis color map

plt.title('Missing Values Heatmap')
plt.show()

Analysis of the output : Education and the values for glycemia seem to be the ones with most missing values. There are no missing values for the 10 year CHD. There are many ways to deal with missing values.

- Remove Rows with Missing Values
- Replace Missing Values with Mean
- Replace Missing Values with Median
- Replace Missing Values with Mode
- Use Forward or Backward Fill

I would prefer either dropping the values since only 10% are missing for glycemia or using median. I don't expect much difference between the two, so I will go with either of them.

In [ ]:
# Handling missing values based on user's choice
def handle_missing_values(df, option):
    """Handle missing values based on user's choice."""
    if option == 'Remove Rows':
        df.dropna(inplace=True)
    elif option == 'Replace with Mean':
        for column in df.columns:
            df[column].fillna(df[column].mean(), inplace=True)
    elif option == 'Replace with Median':
        for column in df.columns:
            df[column].fillna(df[column].median(), inplace=True)
    elif option == 'Replace with Mode':
        for column in df.columns:
            df[column].fillna(df[column].mode()[0], inplace=True)
    elif option == 'Forward or Backward Fill':
        df.fillna(method='ffill', inplace=True)
        df.fillna(method='bfill', inplace=True)
    return df

# User's choice using Google Colab form field dropdown
missing_value_option = 'Replace with Median' #@param ["Remove Rows", "Replace with Mean", "Replace with Median", "Replace with Mode", "Forward or Backward Fill"]

# Handle missing values based on user's choice
df_nm = handle_missing_values(df, missing_value_option)


Let's check again for missing values

In [ ]:
# Checking for missing values
# isnull() returns a DataFrame where each cell is either True or False depending on that cell's null status.
# sum() will then sum the True values (count of missing values) for each column.
print(df_nm.isnull().sum())

# Calculate the percentage of missing values for each column
missing_percentage = (df_nm.isnull().sum() / len(df_nm)) * 100

# Display the percentage of missing values for each column
print(f"Percentage of missing values for each column:\n{missing_percentage}")

# Visualize the missing values as a heatmap
plt.figure(figsize=(10, 6))  # Set the figure size
sns.heatmap(df_nm.isnull(),     # Provide DataFrame with null-status information
            cbar=False,      # Do not draw a color bar
            cmap='viridis')  # Use the viridis color map

plt.title('Missing Values Heatmap')
plt.show()

Analysis of the output : The data is now devoid of missing values.

Let's now check for correlations between these variables

In [ ]:
# Calculate the correlation matrix
correlation_matrix = df.corr()

# Display the correlation matrix
print(correlation_matrix)

# Visualize the correlation matrix as a heatmap
plt.figure(figsize=(12, 8))  # Set the size of the figure
sns.heatmap(correlation_matrix,
            annot=True,  # Annotate each cell with the numeric value
            cmap='coolwarm',  # Use a cool-warm color map
            vmin=-1, vmax=1,  # Set color scale limits
            linewidths=.5)  # Set linewidth between entries in matrix

plt.title('Correlation Matrix')
plt.show()

Analysis of the output : we see some interesting correlations with our dependant variable (target). Mostly age and hypertension hallmarks, which is fairly expectable from a scientific point of view.

## Step 4: Exploratory Data Analysis (EDA) and

Let's explore the dataset to understand it better and figure out how to approach the prediction problem. Visualization helps in identifying patterns and anomalies in the dataset. It will be crucial to deal with imbalanced dataset issues by using techniques like oversampling, undersampling, or SMOTE.


In [ ]:
#Getting information on the data form
df_nm.info()

Output interpretation: The table contains 4,238 entries (or rows) and 16 different categories (or columns) of information. These categories include things like gender (male), age, education, whether the person is a current smoker, and the number of cigarettes smoked per day, among others. All entries in the table are non-null, meaning they all contain data, and the data is in different formats, including integers (int64) and floating-point numbers (float64). The table takes up about 530 kilobytes of memory space.

Let's further explore the data using panda profiling

In [ ]:
pp.ProfileReport(df_nm)

### Overall informations gathered from Panda Profiling :

- There's a strong relationship between the number of cigarettes smoked per day (cigsPerDay) and whether the person is a current smoker (currentSmoker).
- Systolic blood pressure (sysBP) is closely related to diastolic blood pressure (diaBP) and one other field.
- Diastolic blood pressure (diaBP) is also closely related to systolic blood pressure (sysBP) and one other field.
- There's a strong link between glucose levels and diabetes.
- PrevalentHyp is strongly connected with sysBP and one other field.
- The BPMeds, prevalentStroke, and diabetes fields are highly imbalanced, meaning most of the values are the same (80.8% for BPMeds, 94.8% for prevalentStroke, and 82.8% for diabetes).
- Half of the cigsPerDay entries are zero, indicating a lot of non-smokers or missing data : in this case, probably non-smokers since missing data were initial NaNs.

# Imbalance in Data and Why It's an Issue

"Imbalance" in data refers to a situation where the distribution of data among different categories or classes is unequal. One class may have significantly more instances than others.

## Issues Caused by Imbalance:

1. **Model Bias:**
   - An imbalanced dataset can cause a predictive model to be biased towards the majority class, resulting in poor performance for the minority class.

2. **Misleading Accuracy:**
   - A model might show high accuracy by simply predicting the majority class, providing a false sense of effectiveness.

3. **Loss of Information:**
   - Patterns associated with the minority class may be overlooked, leading to a lack of important insights.

4. **Overfitting:**
   - The model may memorize the few available minority class instances rather than generalizing, leading to overfitting.

5. **False Assumptions:**
   - Incorrect assumptions may be made about real-world class distributions, affecting model performance in practical applications.

## Solutions:

1. **Resampling:**
   - Oversample the minority class or undersample the majority class.

2. **Synthetic Data Generation:**
   - Generate new data points for the minority class, e.g., using the Synthetic Minority Over-sampling Technique (SMOTE).

3. **Cost-sensitive Learning:**
   - Penalize the misclassification of the minority class more heavily.

4. **Using Different Evaluation Metrics:**
   - Utilize metrics like the F1-score, precision, recall, or Area Under the Receiver Operating Characteristic (ROC) curve to evaluate model performance.

For the sake of this excercise, let's explore resampling and more specifically SMOTE.



In [ ]:
# Importing Necessary Libraries

# Assumed data loading
# df = pd.read_csv('link_to_dataset')

df_rdy = df_nm.copy()

# Function to handle imbalance
def handle_imbalance(df, column, method):
    if method == "Oversampling":
        df_majority = df[df[column]==0]
        df_minority = df[df[column]==1]
        df_minority_upsampled = resample(df_minority, replace=True, n_samples=len(df_majority), random_state=123)
        df = pd.concat([df_majority, df_minority_upsampled])
    elif method == "Undersampling":
        df_majority = df[df[column]==0]
        df_minority = df[df[column]==1]
        df_majority_downsampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=123)
        df = pd.concat([df_minority, df_majority_downsampled])
    elif method == "SMOTE":
        X = df.drop(columns=[column])
        y = df[column]
        smote = SMOTE(random_state=123)
        X, y = smote.fit_resample(X, y)
        df = pd.concat([X, y], axis=1)
    return df

# Choice of method to handle imbalance for BPMeds
method_BPMeds = 'SMOTE' # @param ["Oversampling", "Undersampling", "SMOTE"]
df_rdy = handle_imbalance(df_rdy, 'BPMeds', method_BPMeds)

# Choice of method to handle imbalance for prevalentStroke
method_prevalentStroke = 'SMOTE' # @param ["Oversampling", "Undersampling", "SMOTE"]
df_rdy = handle_imbalance(df_rdy, 'prevalentStroke', method_prevalentStroke)

# Choice of method to handle imbalance for diabetes
method_diabetes = 'SMOTE' # @param ["Oversampling", "Undersampling", "SMOTE"]
df_rdy = handle_imbalance(df_rdy, 'diabetes', method_diabetes)

df_rdy.head()

Let's explore again the dataset now that those issues have been adressed

In [ ]:
pp.ProfileReport(df_rdy)

Output analysis : The imbalance issues have been solved but there might be other issues that arose. We can try to work those out later (e.g. drop the diabetes variable since it is uniform, remove some features that are highly correlated with each other to reduce multicollinearity), if they end-up being an issue for modelisation. I put the commented code to do so bellow.

In [ ]:
# Calculate the correlation matrix
#corr_matrix = df_rdy.corr().abs()

# Select upper triangle of correlation matrix
#upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.8
#to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]

# Drop the highly correlated features
#df_rdy = df_rdy.drop(df_rdy[to_drop], axis=1)

# Converting 'cigsPerDay' and 'BPMeds' to binary
#df_rdy['cigsPerDay'] = df_rdy['cigsPerDay'].apply(lambda x: 1 if x > 0 else 0)
#df_rdy['BPMeds'] = df_rdy['BPMeds'].apply(lambda x: 1 if x > 0 else 0)

# Optionally drop the 'diabetes' column
# df_rdy = df_rdy.drop(columns=['diabetes'])




## Step 5: Feature Selection and Splitting the Dataset

In this step, we will select the relevant features for our model and split the dataset into training and testing sets. This separation allows us to evaluate the model's performance on unseen data.  

In [ ]:
# Selecting features and target variable
X = df_rdy.drop(columns=['TenYearCHD'])
y = df_rdy['TenYearCHD']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Let's verify if there is any imbalance for target variable before going forward

In [ ]:
print(y_test.unique())
Counter(y_train)

There is a class imbalance in the training dataset: there are significantly more cases of class 0 (18336) than class 1 (7516). This class imbalance can potentially bias the machine learning model towards predicting the majority class (0 in this case), as the model will learn that it can achieve a high accuracy by always predicting the majority class. Let's solve it using one of the same approaches we used before but only on the training dataset. Indeed, applying resampling techniques on the entire dataset and then splitting it could cause data leakage, where information from the testing set leaks into the training set. So we will first split the data into training and testing sets, and then apply the resampling techniques only on the training set.

In [ ]:
# Function to handle imbalance
def handle_imbalance2(X, y, method):
    if method == "Oversampling":
        df = pd.concat([X, y], axis=1)
        df_majority = df[df.TenYearCHD == 0]
        df_minority = df[df.TenYearCHD == 1]
        df_minority_upsampled = resample(df_minority, replace=True, n_samples=len(df_majority), random_state=123)
        df = pd.concat([df_majority, df_minority_upsampled])
        X = df.drop(columns=['TenYearCHD'])
        y = df['TenYearCHD']
    elif method == "Undersampling":
        df = pd.concat([X, y], axis=1)
        df_majority = df[df.TenYearCHD == 0]
        df_minority = df[df.TenYearCHD == 1]
        df_majority_downsampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=123)
        df = pd.concat([df_minority, df_majority_downsampled])
        X = df.drop(columns=['TenYearCHD'])
        y = df['TenYearCHD']
    elif method == "SMOTE":
        smote = SMOTE(random_state=123)
        X, y = smote.fit_resample(X, y)
    return X, y

# Choice of method to handle imbalance
method_unbalvar = 'SMOTE'  # @param ["Oversampling", "Undersampling", "SMOTE"]
X_train, y_train = handle_imbalance2(X_train, y_train, method_unbalvar)

# Verify the new class distributions
print(pd.value_counts(y_train))


The imbalance issue seems to be solved, we can move forward.

## Step 6: Feature Scaling

Standardizing the dataset is crucial for many machine learning models and helps the model to converge faster.


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


## Step 7: Model Building

We will use first the Logistic Regression model for prediction, which is a commonly used algorithm for binary classification problems.




In [ ]:
# Building the Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

## Step 8: Model Evaluation

Evaluate the model performance by comparing the predicted and actual values. We will use metrics such as precision, recall, and accuracy to assess the model performance.



In [ ]:
# Making predictions
y_pred = model.predict(X_test)

# Evaluating the model
print(classification_report(y_test, y_pred))
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

In [ ]:
print('Mdeo Train Score is : ' , model.score(X_train, y_train))
print('Model Test Score is : ' , model.score(X_test, y_test))

Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

# Creating the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Visualizing the confusion matrix
sns.heatmap(cm, annot=True, fmt='g')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


Output analysis :
1. **Precision:**
   - Class 0: \(0.88\): When the model predicts class 0 (no disorder), it is correct \(88\%\) of the time.
   - Class 1: \(0.60\): When the model predicts class 1 (CM disorder), it is correct \(60\%\) of the time.
   - This tells us about the correctness of the model when making a positive prediction.

2. **Recall:**
   - Class 0: \(0.79\): Of the actual class 0 instances, the model correctly predicted \(79\%\) of them.
   - Class 1: \(0.75\): Of the actual class 1 instances, the model correctly predicted \(75\%\) of them.
   - This metric tells us about the model's ability to identify all the relevant cases within a dataset.

3. **F1-Score:**
   - Class 0: \(0.83\): The harmonic mean of precision and recall for class 0 is \(83\%\).
   - Class 1: \(0.66\): The harmonic mean of precision and recall for class 1 is \(66\%\).
   - The F1 Score is a good way to summarize the evaluation of the model with a single number, with 1 being perfect and 0 being the worst.

4. **Support:**
   - Class 0: \(4547\): There are \(4547\) instances of class 0 in the test set.
   - Class 1: \(1917\): There are \(1917\) instances of class 1 in the test set.
   - This gives us an idea about the distribution of classes in the test set.

5. **Accuracy:**
   - \(0.7767\) or \(77.67\%\) is the overall accuracy of the model. This means that \(77.67\%\) of all the predictions were correct.

6. **Macro Avg:**
   - Precision, recall, and F1-score averaged for both classes, without considering the imbalance of classes.
   - Precision: \(0.74\), Recall: \(0.77\), F1-Score: \(0.75\).

7. **Weighted Avg:**
   - Average of the metric, with consideration of class imbalance.
   - Precision: \(0.80\), Recall: \(0.78\), F1-Score: \(0.78\).

So :

- The model performs relatively well, with an overall accuracy of approximately \(77.67\%\).
- The model has higher precision for class 0 compared to class 1, which might be an indication that the model is more reliable when predicting class 0.
- The recall for both classes is fairly comparable, suggesting the model has a decent ability to identify positive instances for both classes.
- There is a significant difference in the F1-score for class 0 and class 1, showing the model has a better balance of precision and recall for class 0.

This is a good start but we surely do better.

# Step 9: Model optimization

Model optimization is the process of finding the best model parameters (hyperparameters) for a given machine learning algorithm to improve its performance. The process involves training the model on different parameter values and evaluating its performance to find the most suitable set of parameters. Different strategies for model optimization include Grid Search, Random Search, and Bayesian Optimization.

### What we will do :

1. **Selection of Optimization Method**:
    - Choose the optimization technique you want to try (Grid Search, Random Search, or Bayesian Optimization). Each method requires different setups and performs the optimization differently:
      - Grid Search: Exhaustively searches over a specified parameter grid.
      - Random Search: Randomly searches over parameters.
      - Bayesian Optimization: Uses probabilistic models to find the minimum of a function.
   
2. **Setting Model Parameters**:
    - Set different hyperparameters for the Logistic Regression model. These include `penalty`, `C`, `solver`, and `max_iter` (see below for details).

3. **Fitting the Model**:
   - Depending on the chosen method, the model will be fitted with different parameters.
      - In Grid Search and Random Search, `GridSearchCV` and `RandomizedSearchCV` are used from `sklearn`.
      - In Bayesian Optimization, `BayesianOptimization` from `bayes_opt` is used.
   
4. **Evaluation**:
    - The best model is evaluated on the test set, and the classification report and accuracy are printed.

### How to Select Parameters:

1. **`penalty`** (regularization term):
   - Options: `l1`, `l2`, `elasticnet`, `none`.
   - Selection guide: Choose `l1` or `l2` for small datasets and `elasticnet` for larger datasets or when you need to use both `l1` and `l2` regularization.
  
2. **`C`** (Inverse of regularization strength):
   - Options: Continuous values, e.g., 1.0.
   - Selection guide: Smaller values specify stronger regularization. Choose values close to zero for strong regularization and higher values for weaker regularization.
  
3. **`solver`** (algorithm to use in the optimization problem):
   - Options: `newton-cg`, `lbfgs`, `liblinear`, `sag`, `saga`.
   - Selection guide: For small datasets, `liblinear` is a good choice, whereas `sag` and `saga` are faster for large datasets. If you chose elasticnet before, you need to chose saga.
  
4. **`max_iter`** (maximum number of iterations taken for the solvers to converge):
   - Options: Integer values, e.g., 100.
   - Selection guide: For faster convergence, use higher values. Monitor the convergence of the model and increase the value if needed.

### Further Tips:

- For `Grid Search`, define a grid for each parameter. The algorithm will evaluate the model performance for each combination of parameters.
- For `Random Search`, define the distribution for each parameter. The algorithm will randomly sample parameters from the distribution.
- For `Bayesian Optimization`, define the range for each parameter. The algorithm will use Bayesian inference to propose better parameters iteratively.
  
In practice, start with a coarse grid or a broader range and then refine the search space based on initial results for more precise optimization.

MANUAL SETTINGS

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

# Form fields for user input
penalty = 'l1'  # @param ['l1', 'l2', 'elasticnet', 'none']
C = 33  # @param {type: "number"}

# Setting solver based on penalty
if penalty == 'l1':
    solver = 'saga'
elif penalty == 'elasticnet':
    solver = 'saga'
else:
    solver = 'lbfgs'


# Fit the model
random_search.fit(X_train, y_train)

# Getting the best parameters
best_params = random_search.best_params_

# Output the best parameters
print(f"The best parameters are: {best_params}")

# Calculate the accuracy on the test set
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Output the accuracy
print(f"The accuracy of the best model on the test data is: {accuracy:.2f}")

Best accuracy I got is 0.78, let's automate this

## GRID SEARCH

In [ ]:
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.01, 0.1, 1, 10],  # Example values
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
if 'elasticnet' in param_grid['penalty']:
    param_grid['l1_ratio'] = [0.1, 0.5, 0.9]  # Example values


In [ ]:
clf = GridSearchCV(LogisticRegression(max_iter=100), param_grid, cv=5, verbose=1, n_jobs=-1)
clf.fit(X_train, y_train)

print(f"Best parameters found: {clf.best_params_}")
print(f"Best cross-validation score: {clf.best_score_:.2f}")


Output analysis : we are not getting much better result, even worse accuracy at 0.76. Let's try another way. Grid search is limiting us to specific values on the grid.

## RANDOM SEARCH

In [ ]:
# Define the range of hyperparameters you want to search over
param_dist = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.001, 0.01, 0.1, 1, 10],
    'l1_ratio': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
}

# Initialize the Logistic Regression model
model = LogisticRegression(solver='saga')

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    model, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1)

# Fit the model
random_search.fit(X_train, y_train)

# Getting the best parameters
best_params = random_search.best_params_

# Output the best parameters
print(f"The best parameters are: {best_params}")

# Calculate the accuracy on the test set using the best model
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Output the accuracy
print(f"The accuracy of the best model on the test data is: {accuracy:.2f}")


Accuracy is slightly improve with random search to 0,78

## BAYESIAN OPTIMIZATION

In [ ]:
!pip install scikit-optimize
from skopt import BayesSearchCV
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load a sample dataset (you can replace this with your dataset)
iris = load_iris()
X, y = iris.data, iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter search space
param_space = {
    'C': (1e-6, 1e+6, 'log-uniform'),  # Search for C values in log scale
    'gamma': (1e-6, 1e+1, 'log-uniform'),  # Search for gamma values in log scale
    'kernel': ['linear', 'rbf'],  # Choose between linear and RBF kernels
}

# Initialize the Bayesian Optimization search
opt = BayesSearchCV(
    SVC(),
    param_space,
    n_iter=50,  # Number of optimization steps
    cv=5,  # Cross-validation folds
    n_jobs=-1,  # Use all available CPU cores
    verbose=1,  # Print progress
    random_state=42,  # Set a random seed for reproducibility
)

# Perform the Bayesian optimization search
opt.fit(X_train, y_train)

# Get the best hyperparameters
best_params = opt.best_params_

# Print the best hyperparameters
print("Best Hyperparameters:", best_params)

# Evaluate the model with the best hyperparameters on the test set
best_model = opt.best_estimator_
test_score = best_model.score(X_test, y_test)

print("Test Accuracy with Best Hyperparameters:", test_score)


So we achieved 1 which is a huge warning sign for overfitting, something that can happen quite often with Bayesian optimization. It's then crucial to assess the model's generalization performance on unseen data to confirm its effectiveness.

It would be also essential to consider other evaluation metrics, such as precision, recall, and F1-score, in addition to accuracy, to get a more comprehensive understanding of model performance.

## Step 9: Comparing different models

Optimization is not getting us much further, either remaining low in accuracy or risking overfitting. Let's evaluate different models for the same dataset and pick the most accurate one using PyCaret. Let's start from the dataset.



In [ ]:
# Fetch the dataset from GitHub
url = "https://raw.githubusercontent.com/wamaw123/Biomedical-Data-Analytics-with-Python/afab193c5cb3d6878755c4d12e8baa821a8ab054/Datasets/23/framingham.csv"
df = pd.read_csv(url)
df.head()

### What is Pycart ?
PyCaret is an open-source Python library designed to simplify and accelerate the end-to-end machine learning and model deployment process. It provides a high-level, easy-to-use interface that automates many of the repetitive tasks and reduces the amount of code required to develop and deploy machine learning models.

### Key features of PyCaret include:
Automated Setup: PyCaret simplifies data preprocessing by automatically handling common tasks like data cleaning, feature selection, and feature engineering. It can also automatically split the data into training and testing sets.

Model Selection: PyCaret allows you to easily compare and evaluate multiple machine learning models, including classification, regression, clustering, and anomaly detection algorithms. It provides a single function to create, train, and evaluate models, making it straightforward to identify the best-performing model for your task.

Hyperparameter Tuning: PyCaret supports hyperparameter tuning for models, helping you find the best combination of hyperparameters to optimize model performance.

Model Interpretability: It provides tools for interpreting and visualizing model results, including feature importance plots, confusion matrices, and ROC curves.

Deployment: PyCaret includes functionality for deploying machine learning models to production, making it easier to transition from model development to real-world applications.

Anomaly Detection: PyCaret also supports anomaly detection, which is useful for identifying rare or unusual data points.

Natural Language Processing (NLP): It has modules for NLP tasks, such as text classification and sentiment analysis.

Time Series Analysis: PyCaret supports time series forecasting and analysis.

Now let's compare models using pycaret

In [ ]:
# Initialize PyCaret setup
setup(data=df, target='TenYearCHD', verbose=False, session_id=123)

# Compare models
compare_models()

 Output analysis :  the Logistic Regression model has the highest accuracy of 0.8510, highest AUC of 0.7201, and reasonable recall and precision. The training time is also very fast at 1.749 seconds.

The other top performers are the LDA, Ridge, and Dummy classifiers, which have slightly lower accuracy than LogisticRegression.

The ensemble methods like Random Forest, AdaBoost, Extra Trees, LightGBM, and XGBoost have good accuracy around 0.84-0.85, but lower AUC and recall/precision than LogisticRegression. Their training times are also slower.

The KNN, SVM, QDA, and Naive Bayes models have decent accuracy between 0.82-0.83, but weaker AUC and recall/precision.

The Decision Tree classifier has much lower performance with 0.7603 accuracy.

Overall, the LogisticRegression model appears to be the best for predicting positive cases based on its top accuracy and AUC, good balance of recall and precision, and fast training time. The ensemble methods are good alternatives, while the other models are outperformed by LogisticRegression.

Let's roll it again but this time after dealing with missing values with median

In [ ]:
# Handling missing values based on user's choice
def handle_missing_values(df, option):
    """Handle missing values based on user's choice."""
    if option == 'Remove Rows':
        df.dropna(inplace=True)
    elif option == 'Replace with Mean':
        for column in df.columns:
            df[column].fillna(df[column].mean(), inplace=True)
    elif option == 'Replace with Median':
        for column in df.columns:
            df[column].fillna(df[column].median(), inplace=True)
    elif option == 'Replace with Mode':
        for column in df.columns:
            df[column].fillna(df[column].mode()[0], inplace=True)
    elif option == 'Forward or Backward Fill':
        df.fillna(method='ffill', inplace=True)
        df.fillna(method='bfill', inplace=True)
    return df

# User's choice using Google Colab form field dropdown
missing_value_option = 'Remove Rows' #@param ["Remove Rows", "Replace with Mean", "Replace with Median", "Replace with Mode", "Forward or Backward Fill"]

# Handle missing values based on user's choice
df_nm = handle_missing_values(df, missing_value_option)


In [ ]:
# Initialize PyCaret setup
setup(data=df_nm, target='TenYearCHD', verbose=False, session_id=123)

# Compare models
compare_models()

Not much difference with or without missing values

Now after dealing with imbalance in the predictors

In [ ]:
df_rdy = df_nm.copy()

# Function to handle imbalance
def handle_imbalance(df, column, method):
    if method == "Oversampling":
        df_majority = df[df[column]==0]
        df_minority = df[df[column]==1]
        df_minority_upsampled = resample(df_minority, replace=True, n_samples=len(df_majority), random_state=123)
        df = pd.concat([df_majority, df_minority_upsampled])
    elif method == "Undersampling":
        df_majority = df[df[column]==0]
        df_minority = df[df[column]==1]
        df_majority_downsampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=123)
        df = pd.concat([df_minority, df_majority_downsampled])
    elif method == "SMOTE":
        X = df.drop(columns=[column])
        y = df[column]
        smote = SMOTE(random_state=123)
        X, y = smote.fit_resample(X, y)
        df = pd.concat([X, y], axis=1)
    return df

# Choice of method to handle imbalance for BPMeds
method_BPMeds = 'SMOTE' # @param ["Oversampling", "Undersampling", "SMOTE"]
df_rdy = handle_imbalance(df_rdy, 'BPMeds', method_BPMeds)

# Choice of method to handle imbalance for prevalentStroke
method_prevalentStroke = 'SMOTE' # @param ["Oversampling", "Undersampling", "SMOTE"]
df_rdy = handle_imbalance(df_rdy, 'prevalentStroke', method_prevalentStroke)

# Choice of method to handle imbalance for diabetes
method_diabetes = 'SMOTE' # @param ["Oversampling", "Undersampling", "SMOTE"]
df_rdy = handle_imbalance(df_rdy, 'diabetes', method_diabetes)

df_rdy.head()

In [ ]:
# Initialize PyCaret setup
setup(data=df_rdy, target='TenYearCHD', verbose=False, session_id=123)

# Compare models
compare_models()

We are reaching high levels of accuracy but we did not consider the risk for imbalance and did not workout the normalization. Let's get that on.

In [ ]:
# Initialize PyCaret setup
setup(data=df_rdy, target='TenYearCHD', verbose=False, session_id=123, normalize=True, transformation=True, transformation_method='yeo-johnson')

# Handle class imbalance using PyCaret's `create_model` function
# You can specify the method as 'SMOTE' or other methods
model = create_model('lr')  # Logistic Regression model

# Compare models
compare_models()


The accuracy is still very high which suggest overfitting again. Let's check optimizing.

In [ ]:
# Initialize PyCaret setup
setup(data=df_rdy, target='TenYearCHD', verbose=False, session_id=123, normalize=True, transformation=True, transformation_method='yeo-johnson')

# Create an Extra Trees Classifier
et_classifier = create_model('et')  # You can adjust other parameters here

# Tune hyperparameters of the Extra Trees Classifier
tuned_et_classifier = tune_model(et_classifier)

# You can also specify a custom hyperparameter grid for tuning:
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
# }
# tuned_et_classifier = tune_model(et_classifier, custom_grid=param_grid)

# Evaluate the tuned model
evaluate_model(tuned_et_classifier)

# You can use this tuned model for predictions
# For example:
# predictions = predict_model(tuned_et_classifier, data=df)


I highly suspect overfitting so let's :

Increase fold to perform more cross validation


In [ ]:
setup(data=df_rdy, target='TenYearCHD', verbose=False, session_id=123, fold=10)  # Increase the number of folds (e.g., fold=10)

# Create an Extra Trees Classifier
et_classifier = create_model('et')  # You can adjust other parameters here

# Tune hyperparameters of the Extra Trees Classifier
tuned_et_classifier = tune_model(et_classifier)

# You can also specify a custom hyperparameter grid for tuning:
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
# }
# tuned_et_classifier = tune_model(et_classifier, custom_grid=param_grid)

# Evaluate the tuned model
evaluate_model(tuned_et_classifier)

# You can use this tuned model for predictions
# For example:
# predictions = predict_model(tuned_et_classifier, data=df)


Now let's test ensemble methods like Bagging or Boosting. These methods can help reduce overfitting by combining multiple models. PyCaret provides functions like ensemble_model to create ensemble models.

In [ ]:
ensemble_model(et_classifier)

A final approach might be to drop some features that might be causing overfitting.

## Step 10: Optimization

In this step, we aim to fine-tune the best model's parameters to enhance its performance using techniques like Grid Search or Random Search.


# Conclusions and perspectives

In this notebook, we